In [1]:
# --- IMPORTS ---
import numpy as np
import stackstac
import pystac_client
import planetary_computer
import dask.array as da
from dask.distributed import Client, LocalCluster
from urllib3.util.retry import Retry
from dask.diagnostics import ProgressBar
import xarray as xr
import matplotlib.pyplot as plt
import ipyleaflet


# --- PARAMETERS ---
# Choose how to group the time dimension: "month", "week", or "year"
time_grouping = "month"  # Options: "month", "week", "year"
spatial_resolution = 50  # Spatial resolution in meters
bands_to_load = ['B02', 'B03', 'B04', 'SCL']  # Blue, Green, Red, Scene Classification
local_cluster = False  # Set to False if using Coiled

In [ ]:

# --- DASK CLUSTER SETUP ---
if local_cluster:
    cluster = LocalCluster()
    client = Client(cluster)
else:
    import coiled
    cluster = coiled.Cluster(name="Timelapse", shutdown_on_close=True)
    cluster.adapt(n_workers=1, maximum=8)
    client = cluster.get_client()


In [ ]:

# --- MAP FOR BOUNDING BOX SELECTION ---
m = ipyleaflet.Map(scroll_wheel_zoom=True)
m.center = (41.64933994767867, -69.94438630063088)
m.zoom = 12
m.layout.height = "500px"
m.layout.width = "500px"
display(m)


In [5]:

# Wait for user to select area, then get bounding box
# (You may want to add a widget for interactive selection in a real workflow)
bounding_box = (m.west, m.south, m.east, m.north)


In [ ]:

# --- LOAD DATA FROM PLANETARY COMPUTER ---
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bounding_box,
    datetime="2024-01-01/2024-12-31"
)
items = search.item_collection()
print(f"Found {len(items)} items in the selected area and time range.")


In [ ]:

# Stack the data using stackstac
data = stackstac.stack(
    items,
    assets=bands_to_load,
    resolution=spatial_resolution,
    epsg=3857,
    bounds_latlon=bounding_box,
)

print("Array size information:")
print(f"Shape: {data.shape}")
print(f"Size in bytes: {data.data.nbytes}")
print(f"Size in GB: {data.data.nbytes / 1e9:.2f} GB")
print(f"Number of chunks: {data.data.npartitions}")
print(f"Chunksize: {(data.data.nbytes / data.data.npartitions) / 1e6:.2f} MB")

data


In [ ]:
client.cluster.scale(6)
print("Scaled the cluster to 6 workers.")

# Establish SCL band on its own
scl_band = data.sel(band='SCL')

# Classes to discard (i.e clouds or defective pixels)
cloudy_classes = [3, 8, 9, 10, 11]

# Cloud mask, i.e where pixels are not cloudy
cloud_mask = scl_band.isin(cloudy_classes)

daily_cloud_mask_frac = cloud_mask.resample(time='1D').mean(dim='time', skipna = True)
daily_cloud_score = daily_cloud_mask_frac.mean(dim=["x", "y"]).compute()
grouping_accessor = getattr(daily_cloud_score.time.dt, time_grouping)

best_daily_timestamps = daily_cloud_score.groupby(grouping_accessor).map(lambda x: x.idxmin())

data_days = data.time.dt.floor('D')

mask_best_days = data_days.isin(best_daily_timestamps)

filtered_data = data.where(mask_best_days, drop=True)

scl_band = filtered_data.sel(band='SCL')

cloud_mask = scl_band.isin(cloudy_classes)

filtered_data = filtered_data.where(~cloud_mask)

rgb_composites = filtered_data.sel(band=['B02', 'B03', 'B04']).resample(time='1d').median(skipna=True, keep_attrs=True)

rgb_composites = rgb_composites.dropna(dim='time', how = 'all')

# Import geogif for creating animated GIFs from geospatial data
import geogif

geogif = geogif.dgif(rgb_composites, fps=4)
geogif.compute()

client.cluster.scale(1)

In [ ]:
geogif.compute()

In [ ]:
# # Compute the 2nd and 98th percentiles for all data in rgb_composites
# global_min = rgb_composites.quantile(0.02).compute()
# global_max = rgb_composites.quantile(0.98).compute()

# # Plot with consistent vmin and vmax across all subplots using percentiles
# rgb_composites.plot.imshow(
#     col="time", 
#     rgb="band", 
#     col_wrap=5,
#     vmin=global_min,
#     vmax=global_max
# )

In [ ]:
# rgb_composites = rgb_composites.ffill("time").bfill("time")

# # Plot with consistent vmin and vmax across all subplots using percentiles
# rgb_composites.plot.imshow(
#     col="time", 
#     rgb="band", 
#     col_wrap=5,
#     vmin=global_min,
#     vmax=global_max
# )